In [ ]:
using Pkg
Pkg.add("Interpolations")
Pkg.add("BasicInterpolators")

# Resolvemos para atmósfera estratificada
Siendo $\vec{q}=\nabla(\tau)$ y $\vec{w}$ el vector velocidad del viento, para una atmósfera estratificada se obtiene $\dfrac{dq_x}{dt}=\dfrac{dq_y}{dt}=0;\dfrac{dq_z}{dt}=-q\dfrac{\partial c}{\partial z}-\vec{q}.\dfrac{\partial \vec{w}}{\partial z}$ (Brouwer 2014).

Implementamos un caso de velocidad de viento constante ($\dfrac{\partial \vec{w}}{\partial z}$) con velocidad del sonido dependiente de la altura con la temperatura: $c_{air}=331 \dfrac{m}{s}\sqrt{\dfrac{T_{z=0m}-\dfrac{z}{300m}}{273.15 K}}$


Obtenemos: $\dfrac{dq_z}{dt}=-\sqrt{q_x^2+q_y^2+q_z^2}\dfrac{\partial c}{\partial z}; met=-\dfrac{331}{600*273.15\sqrt{\dfrac{T_{z=0m}-\dfrac{z}{300m}}{273.15 K}}}=- \dfrac{331^2}{600*273.15*c(T,z)}\dfrac{1}{s} \Rightarrow \dfrac{dq_z}{dt}=\dfrac{1}{q_z}\dfrac{dq_z}{dz}=\dfrac{331^2\sqrt{q_{x}^2+q_{y}^2+q_z^2}}{600*273.15*c(T,z)} \Rightarrow \dfrac{dq_z}{dz}=\dfrac{331^2\sqrt{q_{x}^2+q_{y}^2+q_z^2}q_z}{600*273.15*c(T,z)}$

Con $\dfrac{dt}{d\tau}=1$ y la c.i. $\tau(x,y,z=Source)=0$ obtenemos $\tau=| \int \vec{q} d \vec{s} | \approx \Sigma |\vec{q_i}.\vec{\Delta {s_i}}  |  $ siendo s cada elemento diferencial de la trayectoria recorrida por el rayo acústico.

In [7]:
function c_air_TZ(T_z0,z)
    c_air=331*sqrt((T_z0+273.15-z/300)/273.15) #Introduzco T_Z0 en grados centígrados por comodidad
    return c_air
end

c_air_TZ (generic function with 1 method)

Resolvemos la ODE: $\dfrac{dq_x}{dz}=\dfrac{dq_y}{dz}=0 ;\dfrac{dq_z}{dz}=\dfrac{331^2\sqrt{q_{x}^2+q_{y}^2+q_z^2}q_z}{600*273.15*c(T,z)}$

In [8]:
@time begin
#############################################
using DifferentialEquations

#Argumentos y datos
v_wind=[5.,-10,-5.];
T=20.;  
SourcePoint=[0.,0.,0.];
ReceiverPoint=[5000.,5000.,5000.];

#Definimos la ODE 
    function q!(dq, q, p, z)
        c=c_air_TZ(p[1],z)
        dq[1]=0
        dq[2]=0
        dq[3] = 331^2*sqrt(q[1]^2 + q[2]^2 + q[3]^2)*q[3] / (600 * 273.15 *c)
    end
    #Ángulos iniciales
    theta=rad2deg(acos(1/sqrt(3)))
    phi=45 #Calculados directamente
    # Definimos las condiciones iniciales y los parámetros
    q0=1/(c_air_TZ(T, SourcePoint[3])+(v_wind[1]*cosd(phi)+v_wind[2]*sind(phi))*sind(theta))
    q_x = q0*cosd(phi)*sind(theta)
    q_y = q0*sind(phi)*sind(theta)
    q_z = q0*cosd(theta)
    u0 = [q_x, q_y, q_z]
    

    # Definimps el rango de valores de z. Debemos tener cuidado al optimizar, dado
    # que si algún valor (coordenada z) optimizado está fuera de zspan no podemos evaluarlo.
    zspan = (SourcePoint[3], ReceiverPoint[3])

    # Definimos los parámetros
    p = [T] 

    # Definir el problema ODE
    prob = ODEProblem(q!, u0, zspan, p)
    
    #Resolvemos el problema ODE almacenando en los valores de Z que se corresponden a las uniones de los segmentos.
    #z_values=SegmentsCoords[:,3]
    #, saveat=z_values
    sol = solve(prob, RK4(),dt=1e-3)
    #Mirar documentacion para obtener funcion e introducirla como argumento de entrada
    
    #Guardamos los valores de vec(q)
    qx_vals=zeros(length(sol.u))
    qy_vals=zeros(length(sol.u))
    qz_vals=zeros(length(sol.u))

    for i in 1:length(sol.u)
        qx_vals[i]=sol.u[i][1]
        qy_vals[i]=sol.u[i][2]
        qz_vals[i]=sol.u[i][3]
    end
#########################
end

  0.247176 seconds (92.58 k allocations: 6.269 MiB, 99.24% compilation time: 96% of which was recompilation)


In [9]:
using Interpolations
###### qx y qy son constantes
qx=sum(qx_vals)/length(qx_vals)
######
qy=sum(qy_vals)/length(qy_vals)
######
z_values_linear = collect(range(SourcePoint[3], stop=ReceiverPoint[3], length=length(qz_vals)))
qz_z_linear=LinearInterpolation(z_values_linear, qz_vals)




9-element extrapolate(interpolate((::Vector{Float64},), ::Vector{Float64}, Gridded(Linear())), Throw()) with element type Float64:
 0.0016980037155087805
 0.0016980037252445233
 0.001698003822601959
 0.0016980047961770292
 0.001698014531999111
 0.001698111897358504
 0.0016990862654346487
 0.00170890202527781
 0.0017483714615856465

In [10]:
using DifferentialEquations

function Hamiltonian_l(qx::Float64,qy::Float64,qz_z_linear, SegmentsCoords::Array{Float64, 2})
   
    #Calculamos los traveltime entre cada segmento tomando la media de las q entre cada segmento (aproximación regla Trapecio)
    taus=zeros(length(SegmentsCoords[:,1])-1)
    for i in 1:(length(SegmentsCoords[:,1])-1)
        #qx y qy son constantes, da igual que elemento del vector cojamos. Solo debemos tener cuidado con
        aux_x=(SegmentsCoords[i+1,1]-SegmentsCoords[i,1])
        aux_y=(SegmentsCoords[i+1,2]-SegmentsCoords[i,2])
        # La función de interpolación cúbica puede extrapolarse ligeramente fuera del rango de los datos
        #debido a la naturaleza de la interpolación cúbica.  Iterando la condición inicial lleva a BoundsError. Debo emplear 
        #clamp() para asegurarme que interpola dentro del rango
        z1 = clamp(SegmentsCoords[i,3], SegmentsCoords[1,3], SegmentsCoords[end,3])
        z2 = clamp(SegmentsCoords[i+1,3], SegmentsCoords[1,3], SegmentsCoords[end,3])
        aux_z=(SegmentsCoords[i+1,3]-SegmentsCoords[i,3])
        q_m=sqrt(qx^2+qy^2+(qz_z_linear(z1)+qz_z_linear(z2))^2/4)
        taus[i]=q_m*sqrt(aux_x^2+aux_y^2+aux_z^2)
    end
    
    #Sumamos el travel time de cada segmento para obtener el total
    total_travel_time = sum(taus)
        
    return total_travel_time
end

Hamiltonian_l (generic function with 1 method)

In [11]:
@time begin
using Optim
using Statistics
#Argumentos y datos
v_wind=[5.,-10,-5.];
T=20.;  
SourcePoint=[0.,0.,0.];
ReceiverPoint=[5000.,5000.,5000.];

#Optimizamos las coordenadas de las uniones entre segmentos
function objective(SegmentsCoords)
    # Primero nos aseguramos de que SourcePoint y ReceiverPoint no se optimizan pero que se incluyen en SegmentsCoords
    FullSegmentsCoords = vcat(reshape(SourcePoint, 1, :), SegmentsCoords, reshape(ReceiverPoint, 1, :))
    x12=FullSegmentsCoords[1,2]-FullSegmentsCoords[1,1]
    y12=FullSegmentsCoords[2,2]-FullSegmentsCoords[2,1]
    z12=FullSegmentsCoords[2,3]-FullSegmentsCoords[1,3]
    theta=rad2deg(acos(z12/sqrt(x12^2+y12^2+z12^2)))
    phi=rad2deg(atan(y12,x12)) 
    # Definimos las condiciones iniciales y los parámetros
    q0=1/(c_air_TZ(T, SourcePoint[3])+(v_wind[1]*cosd(phi)+v_wind[2]*sind(phi))*sind(theta))
    q_x = q0*cosd(phi)*sind(theta)
    q_y = q0*sind(phi)*sind(theta)
    q_z = q0*cosd(theta)
    u0 = [q_x, q_y, q_z]
    prob = ODEProblem(q!, u0, zspan, p)
    sol = solve(prob, RK4(),dt=1e-3)
    #Mirar documentacion para obtener funcion e introducirla como argumento de entrada
    
    #Guardamos los valores de vec(q)
    qx_vals=zeros(length(sol.u))
    qy_vals=zeros(length(sol.u))
    qz_vals=zeros(length(sol.u))

    for i in 1:length(sol.u)
        qx_vals[i]=sol.u[i][1]
        qy_vals[i]=sol.u[i][2]
        qz_vals[i]=sol.u[i][3]
    end
    qx=sum(qx_vals)/length(qx_vals)
    ######
    qy=sum(qy_vals)/length(qy_vals)
    ######
    z_values_linear = collect(range(SourcePoint[3], stop=ReceiverPoint[3], length=length(qz_vals)))
    qz_z_linear=LinearInterpolation(z_values_linear, qz_vals)    
    traveltime = Hamiltonian_l(qx, qy, qz_z_linear, FullSegmentsCoords)
    # Calculamos las distancias entre segmentos consecutivos
    segment_distances = diff(FullSegmentsCoords, dims=1)

    # Calculamos la desviación estándar de las distancias
    distance_std_normalized = Statistics.std(segment_distances)/(ReceiverPoint[3]-SourcePoint[3])

    # Agregamos una penalización a la función objetivo basada en la desviación estándar de las distancias
    
    penal = 0
    if x12<10 && y12<10 && z12<10
            penal=1e9
    end
    #Otra penalización si el segundo punto es muy próximo a 0
    return traveltime + distance_std_normalized + penal
end

#Recta troceada desde la fuente al receptor como condición inicial
num_segments=10;
initial_SegmentsCoords = zeros(num_segments-1, 3); 
for i in 1:num_segments-1
    alpha = i / num_segments
    initial_SegmentsCoords[i, :] = SourcePoint + alpha * (ReceiverPoint - SourcePoint) 
end

# Tenemos que tomar restricciones "de caja" para que los valores de z (SegmentsCoords[:,3]) 
#optimizados no esten fuera del intervalo de solución (zspan)
lower_bounds = fill(SourcePoint[3], size(initial_SegmentsCoords))
upper_bounds = fill(ReceiverPoint[3], size(initial_SegmentsCoords))

# Optimizador interno. Limited-memory Broyden-Fletcher-Goldfarb-Shanno
#método de optimización de segundo orden valido para minimizar funciones no lineales.
inner_optimizer = LBFGS()

# Optimizador con las restricciones "de caja"
optimizer_with_constraints = Fminbox(inner_optimizer)

# Optimizamos con las restricciones
result = optimize(objective, lower_bounds, upper_bounds, initial_SegmentsCoords, optimizer_with_constraints)

#Resultado de la optimización
Optimal_SegmentsCoords = result.minimizer;
    
#Incluimos la fuente y el receptor a las coordenadas optimizadas
Full_Optimal_SegmentsCoords= vcat(reshape(SourcePoint, 1, :), Optimal_SegmentsCoords, reshape(ReceiverPoint, 1, :))
    
#Evaluamos el traveltime para la trayectoria optimizada
traveltime =Hamiltonian_l(qx, qy, qz_z_linear, Full_Optimal_SegmentsCoords)
println("The total travel time for the segmented trajectory is: ", traveltime, " seconds")
end

println("Iteraciones realizadas: ", result.iterations)
println("Evaluaciones de la función: ", result.f_calls)



The total travel time for the segmented trajectory is: 25.535505567585158 seconds
  1.964153 seconds (6.77 M allocations: 482.919 MiB, 10.32% gc time, 23.69% compilation time: 72% of which was recompilation)
Iteraciones realizadas: 2
Evaluaciones de la función: 373


In [12]:
Full_Optimal_SegmentsCoords

11×3 Matrix{Float64}:
    0.0        0.0        0.0
   53.5651    11.9427    10.0001
  555.697    531.164    526.799
 1049.11    1029.69    1031.86
 1547.15    1526.45    1536.58
 2036.9     2036.54    2036.5
 2536.95    2536.95    2536.05
 3036.99    3036.99    3036.17
 3537.06    3537.06    3536.21
 4046.94    4046.94    4046.25
 5000.0     5000.0     5000.0

In [13]:
@time begin
using Optim
using Statistics
#Argumentos y datos
v_wind=[5.,-10,-5.];
T=20.;  
SourcePoint=[0.,0.,0.];
ReceiverPoint=[5000.,5000.,5000.];

#Optimizamos las coordenadas de las uniones entre segmentos
function objective(SegmentsCoords)
    # Primero nos aseguramos de que SourcePoint y ReceiverPoint no se optimizan pero que se incluyen en SegmentsCoords
    FullSegmentsCoords = vcat(reshape(SourcePoint, 1, :), SegmentsCoords, reshape(ReceiverPoint, 1, :))
    x12=FullSegmentsCoords[1,2]-FullSegmentsCoords[1,1]
    y12=FullSegmentsCoords[2,2]-FullSegmentsCoords[2,1]
    z12=FullSegmentsCoords[2,3]-FullSegmentsCoords[1,3]
    theta=rad2deg(acos(z12/sqrt(x12^2+y12^2+z12^2)))
    phi=rad2deg(atan(y12,x12)) 
    # Definimos las condiciones iniciales y los parámetros
    q0=1/(c_air_TZ(T, SourcePoint[3])+(v_wind[1]*cosd(phi)+v_wind[2]*sind(phi))*sind(theta))
    q_x = q0*cosd(phi)*sind(theta)
    q_y = q0*sind(phi)*sind(theta)
    q_z = q0*cosd(theta)
    u0 = [q_x, q_y, q_z]
    prob = ODEProblem(q!, u0, zspan, p)
    sol = solve(prob, RK4(),dt=1e-3)
    #Mirar documentacion para obtener funcion e introducirla como argumento de entrada
    
    #Guardamos los valores de vec(q)
    qx_vals=zeros(length(sol.u))
    qy_vals=zeros(length(sol.u))
    qz_vals=zeros(length(sol.u))

    for i in 1:length(sol.u)
        qx_vals[i]=sol.u[i][1]
        qy_vals[i]=sol.u[i][2]
        qz_vals[i]=sol.u[i][3]
    end
    qx=sum(qx_vals)/length(qx_vals)
    ######
    qy=sum(qy_vals)/length(qy_vals)
    ######
    z_values_linear = collect(range(SourcePoint[3], stop=ReceiverPoint[3], length=length(qz_vals)))
    qz_z_linear=LinearInterpolation(z_values_linear, qz_vals)    
    traveltime = Hamiltonian_l(qx, qy, qz_z_linear, FullSegmentsCoords)
    # Calculamos las distancias entre segmentos consecutivos
    segment_distances = diff(FullSegmentsCoords, dims=1)

    # Calculamos la desviación estándar de las distancias
    distance_std_normalized = Statistics.std(segment_distances)/(ReceiverPoint[3]-SourcePoint[3])

    # Agregamos una penalización a la función objetivo basada en la desviación estándar de las distancias
    penal = 0
    if x12<10 && y12<10 && z12<10
            penal=1e9
    end
    #Otra penalización si el segundo punto es muy próximo a 0
    return traveltime + distance_std_normalized + penal
end

#Definimos una matriz de matrices Plot_Coords del tamaño del bucle para luego graficar la distancia de los puntos a una recta
Plot_Coords = Matrix{Matrix{Float64}}(undef, 5, 1)

#Iteramos la condición inicial
initial_SegmentsCoords = zeros(1, 3); #c.i.
for i in 1:5
    if i == 1
        initial_SegmentsCoords[1, :] = SourcePoint + (ReceiverPoint - SourcePoint) / 2
    end
        
    lower_bounds = fill(SourcePoint[3], size(initial_SegmentsCoords))
    upper_bounds = fill(ReceiverPoint[3], size(initial_SegmentsCoords))
    inner_optimizer = LBFGS()
    optimizer_with_constraints = Fminbox(inner_optimizer)
    result = optimize(objective, lower_bounds, upper_bounds, initial_SegmentsCoords, optimizer_with_constraints)
    Optimal_SegmentsCoords = result.minimizer;
    Full_Optimal_SegmentsCoords= vcat(reshape(SourcePoint, 1, :), Optimal_SegmentsCoords, reshape(ReceiverPoint, 1, :))
        
    #Almacenamos las coordenadas de la optimización incluyendo la fuente y el receptor para luego graficar
    Plot_Coords[i]=Full_Optimal_SegmentsCoords
    
    #Evaluamos el travel time para los distintos números de segmentos planteados
    total_time = Hamiltonian_l(qx, qy, qz_z_linear, Full_Optimal_SegmentsCoords)
    println("Num segments: ", Int(size(Full_Optimal_SegmentsCoords, 1) - 1), ". Traveltime:", total_time, " s")
    println("Iteraciones realizadas: ", result.iterations, ". Evaluaciones de la función: ", result.f_calls)
        

    copy_Optimal_SegmentsCoords = zeros(2 * size(Optimal_SegmentsCoords, 1) + 1, 3)
    
    # Calculamos copy_optimal_SegmentsCoords 
    for j in 1:2 * size(Optimal_SegmentsCoords, 1) + 1
        if j==1
            copy_Optimal_SegmentsCoords[j,:]=(SourcePoint+Optimal_SegmentsCoords[1,:])/2
        elseif j==2*size(Optimal_SegmentsCoords,1)+1
            copy_Optimal_SegmentsCoords[j,:]=(Optimal_SegmentsCoords[end,:]+ReceiverPoint)/2
        else
            if iseven(j)
                copy_Optimal_SegmentsCoords[j,:]=Optimal_SegmentsCoords[Int(j/2),:]
            else
                copy_Optimal_SegmentsCoords[j,:]=(Optimal_SegmentsCoords[Int((j-1)/2),:]+Optimal_SegmentsCoords[Int((j+1)/2),:])/2
            end
        end
    end
    #Aquí iteramos la condición inicial
    initial_SegmentsCoords = copy(copy_Optimal_SegmentsCoords)
end
        
end



Num segments: 2. Traveltime:25.56909390849799 s
Iteraciones realizadas: 2. Evaluaciones de la función: 71
Num segments: 4. Traveltime:25.63439248917741 s
Iteraciones realizadas: 6. Evaluaciones de la función: 776
Num segments: 8. Traveltime:25.637624508050294 s
Iteraciones realizadas: 13. Evaluaciones de la función: 1880
Num segments: 16. Traveltime:25.541640730648336 s
Iteraciones realizadas: 5. Evaluaciones de la función: 593
Num segments: 32. Traveltime:25.555003264613788 s
Iteraciones realizadas: 8. Evaluaciones de la función: 1372
 24.813568 seconds (130.60 M allocations: 9.568 GiB, 11.03% gc time, 2.94% compilation time: 35% of which was recompilation)


In [14]:
Plot_Coords[5]

33×3 Matrix{Float64}:
    0.0        0.0             0.0
   15.2853     2.73919e-20    10.0001
   25.6661    20.9244         10.8101
   84.9028    69.7212         51.6563
  144.224    119.421          95.3815
  426.864    361.396         306.46
  707.305    605.282         529.275
  984.795    851.082         761.545
 1256.55    1097.38          998.845
 1313.66    1150.59         1050.75
 1369.35    1203.7          1103.45
 1423.57    1257.1          1156.91
 1477.01    1311.18         1211.72
    ⋮                       
 3334.26    3309.51         3264.79
 3522.75    3509.68         3469.26
 3712.31    3708.58         3672.63
 3903.23    3906.47         3875.09
 4039.89    4046.51         4018.55
 4177.34    4185.82         4161.53
 4315.22    4324.19         4303.8
 4453.72    4462.14         4445.75
 4590.15    4597.17         4584.83
 4726.88    4731.92         4723.63
 4863.66    4866.28         4862.15
 5000.0     5000.0          5000.0